In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

from lib.ECP import Agent, ErgodicCP

In [ ]:
kappa_true = 10
kappa_est = np.random.uniform(1, 100)

lambda_buy = 0.4
lambda_sell = 0.4
q_upper, q_lower = 30, -30
phi = 1e-6

simga = 0.01

T=100
dt=1
K_upper=100
delta0=1e-8

ts = np.linspace(0, T, int(T/dt)+1)

In [ ]:
ErgodicCP(
    lambda_buy=lambda_buy,
    lambda_sell=lambda_sell,
    q_upper=q_upper,
    q_lower=q_lower,
    phi=phi,
    kappa=kappa_true,
).plot_EControl

In [ ]:
Regret_MC = 200
regret_samples = 5


MC_regret = []
MC_kappa = []


for _ in range(Regret_MC):
    regret_list = []
    kappa_list = []
    for _ in range(regret_samples):
        kappa_est = np.random.uniform(50, 100)

        agent = Agent(
            lambda_buy=lambda_buy,
            lambda_sell=lambda_sell,
            q_upper=q_upper,
            q_lower=q_lower,
            phi=phi,
            kappa_est=kappa_est,
            T=T,
            dt=dt,
            K_upper=K_upper,
            delta0=delta0, 
        )

        agent.learning(sigma=simga, kappa_true=kappa_true)

        kappa_list.append(agent.kappa_learnlist)
        regret_list.append(agent.regret(kappa_true=kappa_true))

    kappa_list = np.array(kappa_list)
    regret_list = np.array(regret_list)
    MC_regret.append(np.mean(regret_list, axis=0))
    MC_kappa.append(np.mean(kappa_list, axis=0))

MC_kappa = np.array(MC_kappa)
MC_regret = np.array(MC_regret)

In [ ]:
# df_kappa = pd.DataFrame(MC_kappa)
# df_regret = pd.DataFrame(MC_regret)

# df_kappa.to_csv("file_kappa_t500.csv", header=False, index=False)
# df_regret.to_csv("file_regret_t500.csv", header=False, index=False)

# MC_kappa = pd.read_csv("file_kappa_t100.csv", header=None, index_col=None)
# MC_regret = pd.read_csv("file_regret_t100.csv", header=None, index_col=None)

# MC_kappa = np.array(MC_kappa)
# MC_regret = np.array(MC_regret)

In [ ]:
std = np.std(MC_kappa-kappa_true, axis=0)
ci = 1.64 * std/np.sqrt(len(ts))
y = np.mean(MC_kappa, axis=0)-kappa_true

fig, ax = plt.subplots()
ax.loglog(ts, y)
ax.fill_between(ts, (y-ci), (y+ci), color='b', alpha=.1)

# plt.loglog(ts[1:], ts[1:]**(-0.5), '--')
plt.xlabel("Log Time")
plt.ylabel("Log Error")
plt.title("Log of Learning Error")
plt.show()

In [ ]:
std = np.std(MC_regret, axis=0)
ci = 1.96 * std/np.sqrt(len(ts))
y = np.mean(MC_regret, axis=0)

fig, ax = plt.subplots()
ax.plot(ts, y)
ax.fill_between(ts, (y-ci), (y+ci), color='b', alpha=.1)
plt.xlabel("Time")
plt.ylabel("Regret")
plt.title("Algorithm Regret")
plt.show()

In [ ]:
start = 10

y_axis = (y)[start:]
y_axis_upper = (y+ci)[start:]
x_axis = ts[start:]

# Define the quadratic model
def log_square_model(x, c, m):
    return c * (np.log(x)**2) + m

def calculate_r2(data, fit_data):
    residual = data - fit_data
    ss_res = np.sum(residual**2)
    ss_tot = np.sum((data - np.mean(data))**2)
    r_2 = 1 - ss_res / ss_tot
    return r_2

def log_model(x,c, m):
    return c * np.log(x) + m

# Use log^2 model
params, covariance = curve_fit(log_square_model, x_axis, y_axis)
c_estimated = params[0]
m_estimated = params[1]

params_upper, covariance_upper = curve_fit(log_square_model, x_axis, y_axis_upper)
c_estimated_upper = params_upper[0]
m_estimated_upper = params_upper[1]



# Use log model 
params_log, covariance_log = curve_fit(log_model, x_axis, y_axis)
c_estimated_log = params_log[0]
m_estimated_log = params_log[1]


params_upper_log, covariance_upper_log = curve_fit(log_model, x_axis, y_axis_upper)
c_estimated_upper_log = params_upper_log[0]
m_estimated_upper_log = params_upper_log[1]

In [ ]:
fig, ax = plt.subplots()
ax.plot(ts, y, label='expected regret')
ax.fill_between(ts, (y-ci), (y+ci), color='b', alpha=.1, label='$95\%$ CI of regret')

x_fit = ts[start:]
y_fit = log_square_model(x_fit, c_estimated, m_estimated)
y_fit_upper = log_square_model(x_fit, c_estimated_upper, m_estimated_upper)

y_fit_log = log_model(x_fit, c_estimated_log, m_estimated_log)
y_fit_upper_log = log_model(x_fit, c_estimated_upper_log, m_estimated_upper_log)

ax.plot(x_fit, y_fit, color='mediumslateblue',linestyle=':', label=f'Curve ${np.round(c_estimated, 3)} \ln^2 T + {np.round(m_estimated,4)}, R^2 = {np.round(calculate_r2(y_axis, y_fit),3)}$')
ax.plot(x_fit, y_fit_log, color='violet',linestyle=':', label=f'Curve ${np.round(c_estimated_log, 3)} \ln T   {np.round(m_estimated_log,4)}, R^2 = {np.round(calculate_r2(y_axis, y_fit_log),3)}$')


ax.plot(x_fit, y_fit_upper, color='mediumslateblue',linestyle='dashdot', label=f'Curve ${np.round(c_estimated_upper, 4)} \ln^2 T + {np.round(m_estimated_upper,4)},  R^2 = {np.round(calculate_r2(y_axis_upper, y_fit_upper),3)}$')
ax.plot(x_fit, y_fit_upper_log, color='violet',linestyle='dashdot', label=f'Curve ${np.round(c_estimated_upper_log, 4)} \ln T {np.round(m_estimated_upper_log,4)}, R^2 = {np.round(calculate_r2(y_axis_upper, y_fit_upper_log),3)}$')

plt.xlabel("Time")
plt.ylabel("Regret")
plt.title("Algorithm Regret")
plt.legend()
plt.show()